# Web Scraping with Gemini

### Get the Top 250 IMDb Movies Using Gemini

<img src='./img/movies_portada.png'>

Haciendo uso de la inteligencia artificial, veremos lo sencillo que resulta extraer los datos de una página web. En concreto, usaremos Selenium para acceder a la web y BeautifulSoul para parsear el contenido HTML. Finalmente, le haremos una petición a Gemini para que extraiga los datos requeridos.

### **Consideraciones legales y éticas**

Esta publicación no trata sobre cómo extraer datos de una página web con fines ilegales.

Hay que asegurarse de tener permiso antes de extraer ciertos tipos de datos que puede violar los términos del servicio o incluso regulaciones legales:

- Revise los términos de uso de la página web en relación a los permisos de extracción de datos.
- Priorice el uso de las APIs, si están disponibles, ya que proporcionan acceso legal a los datos.
- Póngase en contacto directamente con el propietario de la página web para comprobar el permiso de extracción de datos.

### **Introducción**

IMDb (Internet Movie Database) es la mayor base de datos que existe en línea, en cuyos comienzos almacenaba información relacionada con películas, pero que con el paso del tiempo, fueron incorporando programas de televisión, eventos, entrega de premios y videojuegos.

Explicaré cómo se puede obtener el listado de las 250 películas con mayor promedio de calificaciones, basado en votos de los usuarios registrados en el sitio web.

Para ello, primero usaremos Selenium para acceder a la web y BeautifulSoul para parsear el contenido HTML. Finalmente, le haremos una petición a Gemini para que extraiga los datos requeridos.

### **Instalación de dependencias**

Me encuentro en un “entorno conda” en la que tengo instalada la versión 3.12.2 de Python.

A continuación, se muestran las librerías necesarias para la ejecución del código creado:

- ipykernel : kernel de Jupyter necesario para ejecutar Python.
- tqdm : herramienta útil para crear barras de progreso en bucles.
- joblib : biblioteca que permite la serialización de objetos Python.
- scipy : biblioteca para el cálculo avanzado que permite manipular y visualizar datos de alto nivel.
- bs4 : librería BeautifulSoup que permite extraer datos de HTML y XML.
- pandas : biblioteca escrita sobre Numpy para la manipulación y el análisis de datos.
- selenium: herramienta de automatización web que nos permite interactuar con páginas web.
- webdriver_manager: biblioteca que permite gestionar automáticamente los controladores de diferentes navegadores.
- google-generativeai: biblioteca y herramientas de cliente API de alto nivel de Google Generative AI (Gemini).

Ejecutamos en la terminal el siguiente código:

```
pip install ipykernel tqdm joblib scipy pandas bs4 selenium webdriver_manager google-generativeai
```

En mi caso, también he tenido que actualizar "ipywidgets" y "jupiter"

```
pip install ipywidgets --upgrade
pip install jupyter --upgrade
```

In [1]:
#%pip install ipykernel tqdm joblib pandas bs4 selenium webdriver_manager google-generativeai
#%pip install ipywidgets --upgrade
#%pip install jupyter --upgrade

### **Importación de dependencias**

In [4]:
import csv
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import Markdown
import google.generativeai as genai
from google.generativeai.types import ContentType

### **Configuración del navegador**

In [2]:
# Set up Chrome options
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-first-run") # Desactiva la pantalla de bienvenida
chrome_options.add_argument("--no-default-browser-check") # Desactiva la comprobación que hace chrome por defecto al iniciarse
chrome_options.add_argument("user-data-dir=C:/Users/jagui/AppData/Local/Google/Chrome/User Data/Profile 1") # ruta de un perfil preestablecido
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Required for some environments
chrome_options.add_argument("--window-size=1920x1080")  # Set window size to ensure visibility

### **Abrir el navegador**

In [3]:
# Configurar el service con ChromeDriverManager
service = Service(ChromeDriverManager().install())

# Crear una nueva instancia del webDriver de Chrome con las opciones configuradas 
driver = webdriver.Chrome(service=service, options=chrome_options)

# Abrir la página web
url = "https://www.imdb.com/chart/top/"
driver.get(url)

El navegador se abre y navega directamente a la url que le hemos indicado

<img src='./img/top_imdb.jpg' width=800>

En la imagen vemos toda la información que podemos extraer: 

- título de la película
- año de estreno
- duración de la película
- número de votaciones
- calificación

### **Parseo el HTML, extraigo el contenido y cierro el navegador**

In [5]:
# Parsear el HTML con BeautifulSoup
content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

# Extraer el contenido HTML y lo almaceno como un str
html_content = soup.get_text()

time.sleep(2)

# Cerrar el navegador
driver.quit()

### **Configurar Gemini**

**Generar una una clave API**

Lo primero que hay que hacer es obtener tu [clave API de Google AI for Developers](https://aistudio.google.com/app/apikey), siguiendo los pasos que nos van indicando.

**Modelos disponibles a través de la API libre**

In [7]:
# Introduce tu API_KEY
GOOGLE_API_KEY = 'TU_API_KEY'

# Configurar la API
genai.configure(api_key = GOOGLE_API_KEY)

In [8]:
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(model.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


Utilizo uno de los modelos disponibles. Por ejemplo el modelo 'gemini-1.5-pro'. Veamos qué características tiene:

In [9]:
for model in genai.list_models():
    if model.name == 'models/gemini-1.5-pro':
        print(model) 
        break  

Model(name='models/gemini-1.5-pro',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Pro',
      description='Mid-size multimodal model that supports up to 2 million tokens',
      input_token_limit=2097152,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)


**Definir el modelo**

In [10]:
modelo = genai.GenerativeModel('gemini-1.5-pro')

### **Definir el prompt y llamar a la API**

In [11]:
text_prompt = '''Quiero que generes un CSV con las 250 películas que aparecen. \n
El CSV debe contener: título de la película, año de estreno, duración de la película, calificación.'''

# poner el contenido HTML y la petición que le hacemos al modelo en una lista
prompt = [html_content, text_prompt]

# hacemos la petición al modelo y almacenamos la respuesta
response = modelo.generate_content(prompt)

# Para evitar tener la salida en formato Markdown, vamos a utilizar la herramienta IPython Markdown
Markdown(response.text)

```csv
titulo,año,duracion,calificacion
Cadena perpetua,1994,2h 22m,9.3
El padrino,1972,2h 55m,9.2
El caballero oscuro,2008,2h 32m,9.0
El padrino parte II,1974,3h 22m,9.0
12 hombres sin piedad,1957,1h 36m,9.0
La lista de Schindler,1993,3h 15m,9.0
El señor de los anillos: El retorno del rey,2003,3h 21m,9.0
Pulp Fiction,1994,2h 34m,8.9
El señor de los anillos: La comunidad del anillo,2001,2h 58m,8.9
El bueno, el feo y el malo,1966,3h 2m,8.8
Forrest Gump,1994,2h 22m,8.8
El señor de los anillos: Las dos torres,2002,2h 59m,8.8
El club de la lucha,1999,2h 19m,8.8
Origen,2010,2h 28m,8.8
El imperio contraataca,1980,2h 4m,8.7
Matrix,1999,2h 16m,8.7
Uno de los nuestros,1990,2h 25m,8.7
Alguien voló sobre el nido del cuco,1975,2h 13m,8.7
Interstellar,2014,2h 49m,8.7
Seven,1995,2h 7m,8.6
¡Qué bello es vivir!,1946,2h 10m,8.6
Los siete samuráis,1954,3h 27m,8.6
El silencio de los corderos,1991,1h 58m,8.6
Salvar al soldado Ryan,1998,2h 49m,8.6
Ciudad de Dios,2002,2h 10m,8.6
La vida es bella,1997,1h 56m,8.6
La milla verde,1999,3h 9m,8.6
Terminator 2: El juicio final,1991,2h 17m,8.6
La guerra de las galaxias,1977,2h 1m,8.6
Regreso al futuro,1985,1h 56m,8.5
El viaje de Chihiro,2001,2h 4m,8.6
El pianista,2002,2h 30m,8.5
Parásitos,2019,2h 12m,8.5
Psicosis,1960,1h 49m,8.5
Gladiator (El gladiador),2000,2h 35m,8.5
El rey león,1994,1h 28m,8.5
Spider-Man: Cruzando el multiverso,2023,2h 20m,8.5
Infiltrados,2006,2h 31m,8.5
Whiplash,2014,1h 46m,8.5
American History X,1998,1h 59m,8.5
El profesional (Léon),1994,1h 50m,8.5
Hotaru no haka,1988,1h 28m,8.5
El truco final (El prestigio),2006,2h 10m,8.5
Harakiri,1962,2h 13m,8.6
Dune: Parte dos,2024,2h 46m,8.5
Sospechosos habituales,1995,1h 46m,8.5
Casablanca,1942,1h 42m,8.5
Intocable,2011,1h 52m,8.5
Cinema Paradiso,1988,2h 54m,8.5
Tiempos modernos,1936,1h 27m,8.5
Alien, el octavo pasajero,1979,1h 57m,8.5
La ventana indiscreta,1954,1h 52m,8.5
Hasta que llegó su hora,1968,2h 46m,8.5
Luces de la ciudad,1931,1h 27m,8.5
Django desencadenado,2012,2h 45m,8.5
Apocalypse Now,1979,2h 27m,8.4
Memento,2000,1h 53m,8.4
WALL·E,2008,1h 38m,8.4
En busca del arca perdida,1981,1h 55m,8.4
12th Fail,2023,2h 27m,8.4
La vida de los otros,2006,2h 17m,8.4
El crepúsculo de los dioses,1950,1h 50m,8.4
Vengadores: Infinity War,2018,2h 29m,8.4
Senderos de gloria,1957,1h 28m,8.4
Spider-Man: Un nuevo universo,2018,1h 57m,8.4
Testigo de cargo,1957,1h 56m,8.4
El resplandor,1980,2h 26m,8.4
El gran dictador,1940,2h 5m,8.4
Aliens: El regreso,1986,2h 17m,8.4
Malditos bastardos,2009,2h 33m,8.4
El caballero oscuro: La leyenda renace,2012,2h 44m,8.4
Coco,2017,1h 45m,8.4
Amadeus,1984,2h 40m,8.4
Toy Story,1995,1h 21m,8.3
¿Teléfono rojo? Volamos hacia Moscú,1964,1h 35m,8.3
Old Boy,2003,2h,8.3
American Beauty,1999,2h 2m,8.3
Vengadores: Endgame,2019,3h 1m,8.4
El submarino (Das Boot),1981,2h 29m,8.4
Braveheart,1995,2h 58m,8.3
El indomable Will Hunting,1997,2h 6m,8.3
La princesa Mononoke,1997,2h 13m,8.3
Joker,2019,2h 2m,8.4
Your Name.,2016,1h 46m,8.4
El infierno del odio,1963,2h 23m,8.4
3 Idiots,2009,2h 50m,8.4
Érase una vez en América,1984,3h 49m,8.3
Cantando bajo la lluvia,1952,1h 43m,8.3
Cafarnaúm,2018,2h 6m,8.4
Masacre (Ven y mira),1985,2h 22m,8.4
Réquiem por un sueño,2000,1h 42m,8.3
Toy Story 3,2010,1h 43m,8.3
El retorno del jedi,1983,2h 11m,8.3
¡Olvídate de mí!,2004,1h 48m,8.3
La caza,2012,1h 55m,8.3
2001: Una odisea del espacio,1968,2h 29m,8.3
Vivir,1952,2h 23m,8.3
Reservoir Dogs,1992,1h 39m,8.3
Lawrence de Arabia,1962,3h 38m,8.3
El apartamento,1960,2h 5m,8.3
Incendios,2010,2h 11m,8.3
Con la muerte en los talones,1959,2h 16m,8.3
El precio del poder,1983,2h 50m,8.3
Oppenheimer,2023,3h,8.3
Ciudadano Kane,1941,1h 59m,8.3
Perdición,1944,1h 47m,8.3
M, el vampiro de Düsseldorf,1931,1h 57m,8.3
Vértigo (De entre los muertos),1958,2h 8m,8.3
La chaqueta metálica,1987,1h 56m,8.2
Heat,1995,2h 50m,8.3
Amélie,2001,2h 2m,8.3
Up,2009,1h 36m,8.3
La naranja mecánica,1971,2h 16m,8.2
Matar a un ruiseñor,1962,2h 9m,8.3
Nader y Simin, una separación,2011,2h 3m,8.3
El golpe,1973,2h 9m,8.3
Jungla de cristal,1988,2h 12m,8.2
Indiana Jones y la última cruzada,1989,2h 7m,8.2
Taare Zameen Par,2007,2h 42m,8.3
Metrópolis,1927,2h 33m,8.3
Snatch. Cerdos y diamantes,2000,1h 44m,8.2
1917,2019,1h 59m,8.2
L.A. Confidential,1997,2h 18m,8.2
Ladrón de bicicletas,1948,1h 29m,8.3
Hamilton,2020,2h 40m,8.3
El hundimiento,2004,2h 36m,8.2
Taxi Driver,1976,1h 54m,8.2
Dangal,2016,2h 41m,8.3
Batman Begins,2005,2h 20m,8.2
La muerte tenía un precio,1965,2h 12m,8.2
El lobo de Wall Street,2013,3h,8.2
Green Book,2018,2h 10m,8.2
Con faldas y a lo loco,1959,2h 1m,8.2
El chico,1921,1h 8m,8.2
El show de Truman,1998,1h 43m,8.2
Vencedores o vencidos,1961,2h 59m,8.3
El padre,2020,1h 37m,8.2
Eva al desnudo,1950,2h 18m,8.2
Shutter Island,2010,2h 18m,8.2
Pozos de ambición,2007,2h 38m,8.2
Jurassic Park (Parque Jurásico),1993,2h 7m,8.2
Top Gun: Maverick,2022,2h 10m,8.2
Casino,1995,2h 58m,8.2
Ran,1985,2h 40m,8.2
El sexto sentido,1999,1h 47m,8.2
El laberinto del fauno,2006,1h 58m,8.2
Sin perdón,1992,2h 10m,8.2
No es país para viejos,2007,2h 2m,8.2
La cosa (El enigma de otro mundo),1982,1h 49m,8.2
Una mente maravillosa,2001,2h 15m,8.2
Kill Bill: Volumen 1,2003,1h 51m,8.2
El tesoro de Sierra Madre,1948,2h 6m,8.2
Yojimbo,1961,1h 50m,8.2
La gran evasión,1963,2h 52m,8.2
Los caballeros de la mesa cuadrada y sus locos seguidores,1975,1h 31m,8.2
Buscando a Nemo,2003,1h 40m,8.2
Prisioneros,2013,2h 33m,8.2
El castillo ambulante,2004,1h 59m,8.2
Rashomon,1950,1h 28m,8.2
El hombre elefante,1980,2h 4m,8.2
Crimen perfecto,1954,1h 45m,8.2
Chinatown,1974,2h 10m,8.1
Lo que el viento se llevó,1939,3h 58m,8.2
Lock & Stock,1998,1h 47m,8.1
El secreto de sus ojos,2009,2h 9m,8.2
V de vendetta,2005,2h 12m,8.1
Del revés (Inside Out),2015,1h 35m,8.1
Tres anuncios en las afueras,2017,1h 55m,8.1
Toro salvaje,1980,2h 9m,8.1
Trainspotting,1996,1h 33m,8.1
El puente sobre el río Kwai,1957,2h 41m,8.1
Klaus,2019,1h 36m,8.2
Atrápame si puedes,2002,2h 21m,8.1
Fargo,1996,1h 38m,8.1
Warrior,2011,2h 20m,8.1
Spider-Man: No Way Home,2021,2h 28m,8.2
Gran Torino,2008,1h 56m,8.1
Harry Potter y las Reliquias de la Muerte: Parte 2,2011,2h 10m,8.1
Million Dollar Baby,2004,2h 12m,8.1
Mi vecino Totoro,1988,1h 26m,8.1
Mad Max: Furia en la carretera,2015,2h,8.1
Children of Heaven,1997,1h 29m,8.2
Ben-Hur,1959,3h 32m,8.1
12 años de esclavitud,2013,2h 14m,8.1
Barry Lyndon,1975,3h 5m,8.1
Antes de amanecer,1995,1h 41m,8.1
Blade Runner,1982,1h 57m,8.1
El gran hotel Budapest,2014,1h 39m,8.1
Hasta el último hombre (Hacksaw Ridge),2016,2h 19m,8.1
Perdida,2014,2h 29m,8.1
El club de los poetas muertos,1989,2h 8m,8.1
Maharaja,2024,2h 21m,8.1
Memories of Murder (Crónica de un asesino en serie),2003,2h 11m,8.1
En el nombre del padre,1993,2h 13m,8.1
La quimera del oro,1925,1h 35m,8.1
Monstruos, S.A.,2001,1h 32m,8.1
Relatos salvajes,2014,2h 2m,8.1
El cazador,1978,3h 3m,8.1
El maquinista de La General,1926,1h 18m,8.1
El moderno Sherlock Holmes,1924,45m,8.2
Tiburón,1975,2h 4m,8.1
Ratatouille,2007,1h 51m,8.1
How to Train Your Dragon,2010,1h 38m,8.1
La ley del silencio,1954,1h 48m,8.1
Mary and Max,2009,1h 32m,8.1
El salario del miedo,1953,2h 11m,8.1
Fresas salvajes,1957,1h 32m,8.1
El tercer hombre,1949,1h 44m,8.1
Le Mans '66,2019,2h 32m,8.1
Caballero sin espada,1939,2h 9m,8.1
Logan,2017,2h 17m,8.1
Cuentos de Tokio,1953,2h 16m,8.1
Rocky,1976,2h,8.1
El gran Lebowski,1998,1h 57m,8.1
El séptimo sello,1957,1h 36m,8.1
La habitación,2015,1h 58m,8.1
Spotlight,2015,2h 9m,8.1
Terminator,1984,1h 47m,8.1
Hotel Rwanda,2004,2h 1m,8.1
Platoon,1986,2h,8.1
El odio,1995,1h 38m,8.1
Piratas del Caribe: La maldición de la Perla Negra,2003,2h 23m,8.1
Antes del atardecer,2004,1h 20m,8.1
La pasión de Juana de Arco,1928,1h 50m,8.1
Jai Bhim,2021,2h 44m,8.7
Los mejores años de nuestra vida,1946,2h 50m,8.1
El exorcista,1973,2h 2m,8.1
Rush,2013,2h 3m,8.1
Los Increíbles,2004,1h 55m,8.0
El mago de Oz,1939,1h 42m,8.1
Network, un mundo implacable,1976,2h 1m,8.1
Cuenta conmigo,1986,1h 29m,8.1
Siempre a tu lado (Hachiko),2009,1h 33m,8.1
Sonrisas y lágrimas,1965,2h 52m,8.1
Mi padre y mi hijo,2005,1h 52m,8.2
La doncella,2016,2h 25m,8.1
Ser o no ser,1942,1h 39m,8.1
Hacia rutas salvajes,2007,2h 28m,8.1
La batalla de Argel,1966,2h 1m,8.1
Las uvas de la ira,1940,2h 9m,8.1
Atrapado en el tiempo,1993,1h 41m,8.0
El gigante de hierro,1999,1h 26m,8.1
Amores perros,2000,2h 34m,8.0
Rebeca,1940,2h 10m,8.1
Criadas y señoras,2011,2h 26m,8.1
La leyenda del indomable,1967,2h 7m,8.1
Sucedió una noche,1934,1h 45m,8.1
Paris, Texas,1984,2h 25m,8.1
Drishyam,2015,2h 43m,8.2
Guardianes de la noche - Kimetsu no Yaiba - La película: Tren Infinito,2020,1h 57m,8.2
``` 


A tener en cuenta:

- Rara vez ocurre, pero en el caso de que aparezca un error con código 500, hay que volver a ejecutar el código anterior (incluso varias veces), ya que se trata de errores internos del servidor, que suelen ser temporales.
- Hay que procurar no hacer peticiones con demasiada frecuencia, porque sino la página web impondrá limitaciones. En este caso, si hacemos varias peticiones muy seguidas, IMDB, en vez de devolvernos las 250 películas, nos devolverá menos.
- En el caso de que el modelo te devuelva una respuesta que no es la que esperas, deberás modificar el prompt y poner el adecuado.

### **Preparamos los datos para guardarlo en un csv**

Ya tengo una respuesta en formato csv. Ahora ya sólo hay que escribir el código para guardar este texto en un archivo.

Pero antes tengo que hacer unas pequeñas transformaciones en el texto para poder guardarlo:

- Eliminar espacios en blanco al inicio y al final.
- Hay títulos de películas que están divididas por una coma ', '. Tenemos que reemplazarlo por ': ', porque sino al guardar el CSV, creerá que se trata de diferentes campos y no lo guardará correctamente.

In [12]:
# Eliminar espacios en blanco al inicio y al final
# Sino lo hago, al guardar el csv, se va a confundir con la coma
response_csv =response.text.strip().replace(', ', ': ')

In [15]:
# Abrir un archivo CSV en modo escritura
with open('peliculas_top250_imdb.csv', 'w', newline='') as csvfile:
    # Crea un objeto escritor de CSV
    csvwriter = csv.writer(csvfile)

    # Escribe cada línea del CSV generado por Gemini en el archivo
    for row in response_csv.splitlines()[1:-1]:
        csvwriter.writerow(row.split(','))

print("El archivo CSV 'peliculas_top250_imdb.csv' ha sido creado exitosamente.")

El archivo CSV 'peliculas_top250_imdb.csv' ha sido creado exitosamente.


In [16]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('peliculas_top250_imdb.csv')
df

,titulo,año,duracion,calificacion
0,Cadena perpetua,1994,2h 22m,9.3
1,El padrino,1972,2h 55m,9.2
2,El caballero oscuro,2008,2h 32m,9.0
3,El padrino parte II,1974,3h 22m,9.0
4,12 hombres sin piedad,1957,1h 36m,9.0
...,...,...,...,...
245,La leyenda del indomable,1967,2h 7m,8.1
246,Sucedió una noche,1934,1h 45m,8.1
247,Paris: Texas,1984,2h 25m,8.1
248,Drishyam,2015,2h 43m,8.2


### **Conclusiones**

El proceso de web scraping aplicado en este artículo demuestra cómo herramientas como Selenium y BeautifulSoup permiten automatizar la extracción de datos de sitios web dinámicos como IMDb. Este enfoque tiene varias ventajas y desafíos que deben tenerse en cuenta:

1. *Versatilidad de Selenium*: Utilizar Selenium permite interactuar con sitios web que utilizan JavaScript para cargar contenido dinámicamente. Esto es esencial en plataformas como IMDb, donde ciertos datos, como reseñas o detalles sobre películas, pueden no estar disponibles inmediatamente en el HTML inicial. Además, Selenium facilita el acceso a páginas protegidas o que requieren navegación compleja.

2. *Eficiencia de BeautifulSoup*: BeautifulSoup permite una navegación rápida y eficiente del contenido HTML una vez cargado. Al combinarlo con Selenium, se obtiene lo mejor de ambos mundos: la capacidad de cargar páginas dinámicas y luego procesar el contenido de manera eficiente.

3. *Peticiones a Gemini para análisis*: La solicitud a Gemini para extraer los datos requeridos añade un valor adicional, ya que permite automatizar el análisis de datos obtenidos, lo cual es especialmente útil cuando se buscan patrones o se trabaja con grandes cantidades de información.

4. *Consideraciones éticas y legales*: Es importante mencionar que cualquier actividad de web scraping debe cumplir con las políticas de uso de las plataformas y las normativas legales correspondientes, respetando los límites impuestos por las APIs o los Términos de Servicio del sitio web en cuestión.

5. *Retos de Mantenimiento*: Un desafío inherente a los proyectos de scraping es la fragilidad del código ante cambios en la estructura de la página. Dado que IMDb, como otros sitios web, puede modificar su diseño o estructura de HTML, el código de scraping puede necesitar ajustes regulares.

6. *Aplicaciones Prácticas*: Esta técnica es muy útil para extraer información de valor en investigaciones, análisis de datos o proyectos personales, como la creación de bases de datos personalizadas de películas, actores o reseñas de cine, lo que demuestra el potencial de usar estas herramientas para fines específicos.

Estas conclusiones refuerzan la importancia de la combinación de tecnologías como Selenium y BeautifulSoup para manejar sitios web dinámicos y la versatilidad de las herramientas de análisis de datos como Gemini.

### **Referencias**

- [IMDb](https://www.imdb.com/)
- [Python](https://www.python.org/)
- [Ipykernel](https://pypi.org/project/ipykernel/)
- [tqdm](https://pypi.org/project/tqdm/)
- [joblib](https://pypi.org/project/joblib/)
- [Scipy](https://pypi.org/project/scipy/)
- [Pandas](https://pandas.pydata.org/)
- [BeautifulSoup](https://pypi.org/project/beautifulsoup4/)
- [Selenium](https://www.selenium.dev/)
- [webdriver-manager](https://pypi.org/project/webdriver-manager/)
- [google-generativeai](https://pypi.org/project/google-generativeai/)